# 如何使用crawler_tool爬取資料


在本篇中你會瞭解到:
1. 如何使用crawler_tool內建函數抓取股價資料
2. 如何使用crawler_tool內建的工具建立新的爬蟲函數


***
## 爬取股價資料
下面是一個簡單的例子，當你想爬取台泥(1101)2022年1月的股票時可以這麼做:

In [9]:
from crawler_tool import get_month_price     

month_df = get_month_price(
    stock_id='1101',        # 股票名稱
    year=2022,              # 年份
    month=4                 # 月份
)
month_df.head()

,date,volume,value,open,high,low,close,record,stock_id
0,2022-04-01,9917831,493504834,49.70,50.00,49.45,49.75,7457,1101
1,2022-04-06,10985331,548362617,49.75,50.20,49.60,50.20,5728,1101
2,2022-04-07,11245643,559937105,49.80,50.20,49.50,49.55,7769,1101
3,2022-04-08,6494233,322457815,49.60,49.80,49.40,49.80,4598,1101
4,2022-04-11,8089418,400306570,49.80,49.95,49.10,49.45,8178,1101


其中*year*和*month*參數也可以使用str:

In [10]:
str_month_df = get_month_price(
    stock_id='1101',            # 股票名稱
    year='2022',                # 年份
    month='4'                   # 月份
)
str_month_df.head()

,date,volume,value,open,high,low,close,record,stock_id
0,2022-04-01,9917831,493504834,49.70,50.00,49.45,49.75,7457,1101
1,2022-04-06,10985331,548362617,49.75,50.20,49.60,50.20,5728,1101
2,2022-04-07,11245643,559937105,49.80,50.20,49.50,49.55,7769,1101
3,2022-04-08,6494233,322457815,49.60,49.80,49.40,49.80,4598,1101
4,2022-04-11,8089418,400306570,49.80,49.95,49.10,49.45,8178,1101


而如果是想爬取日資料你可以使用*get_day_price*函數:

In [2]:
from crawler_tool import get_day_price

day_df = get_day_price(
    stock_id='1101',        # 股票名稱
    year=2022,              # 年份
    month=4                 # 月份
    day=1                   # 日期
)
day_df

,date,volume,value,open,high,low,close,record,stock_id
0,2022-04-01,9917831,493504834,49.7,50.0,49.45,49.75,7457,1101


當爬蟲爬取成功時，會回傳DataFrame型態的股價資料。

而當爬蟲抓取失敗時，爬蟲會回傳預設的值(None)，如果你想自己決定回傳值，你可以使用*fail_return*參數:

In [6]:
fail_return = get_day_price(
    stock_id='0000', # 不存在的股票
    year='2000',
    month='12',
    day='21',
    fail_return='crawler fail'
)
fail_return

'crawler fail'

***
### 錯誤發生的條件

有幾種情況會使得爬蟲抓取失敗:
1. 不存在相關的資料，比如抓取了不存在的股票號碼。
2. 爬取的請求被網站拒絕，可能是爬取的時間間隔過短，後面將介紹如何調整。
3. 格式解析失敗，網站回傳的資料爬蟲無法解析成正確的格式。

不管是何種情況發生時，爬蟲都會在當前目錄下生成紀錄該錯誤log文件。
如果是第1,2種錯誤會保存到stock_crawler_error.log，而格式解析失敗的錯誤會保存到format_error.log文件中。

***
### 爬蟲的執行步驟

而當錯誤發生時，爬蟲並不會立刻結束。爬蟲會進行第二次的爬取，直到成功或是已經爬取了指定的次數。一般而言，爬蟲最多會爬取5次，每次之間會進行3秒的間隔時間。
爬蟲的爬取會遵循以下幾個步驟:
1. 如果已經爬取了5次則直接結束
2. 爬取資料
3. 如果爬取失敗，則紀錄錯誤到指定的log文件中，並到(7)
4. 對資料進行格式解析
5. 如果解析失敗，則紀錄錯誤到指定的log文件中，並到(7)
6. 回傳解析後的資料
7. 暫停3秒後回到(1)

到這裡你已經會使用crawler_tool的爬蟲函數抓取資料了，下面會介紹crawler_tool有哪些工具可以幫助你實做出更多的爬蟲函數。

***
## crawler_tool包內建的工具

首先我們先來看看crawler_tool包裡面有哪些檔案，以及他們在包裡扮演了什麼角色:
* crawlers.py: 提供了大量爬蟲函數
* crawler_decorator.py: 提供了輔助爬蟲函數的工具
* twse_stock_day_crawler.py: 基於前面兩個工具的爬蟲實作

下面我們將一個一個介紹。

***

### crawlers.py

該檔案中內建了許多爬蟲函數可以爬取資料

比如你可以使用*get_latest_trading_days*來取得最近的一次交易日的日期:

In [12]:
from crawler_tool import get_latest_trading_days

get_latest_trading_days()

'20220426'

但有一點是需要注意的，那就是使用crawlers.py裡面的爬蟲在報錯時並不會像前面介紹的get_month_price那樣紀錄錯誤後重跑個5次。而是直接報錯，然後程式停止。
之所以有這樣的不同是因為「紀錄錯誤後重跑個5次」這功能是實作在crawler_decorator.py中的。

而*get_month_price*正是使用了crawlers.py中的*get_stock_month_price*函數加上crawler_decorator.py中紀錄報錯功能實現的，你甚至可以呼叫看看原始輸出長什麼樣。

In [58]:
import crawler_tool.crawlers as crawlers

month_price=crawlers.get_stock_month_price(
    stock_id='1101',
    year='2022',
    month='01'
)
month_price

{'stat': 'OK',
 'date': '20220101',
 'title': '111年01月 1101 台泥             各日成交資訊',
 'fields': ['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數'],
 'data': [['111/01/03',
   '12,539,636',
   '597,825,872',
   '48.05',
   '48.15',
   '47.35',
   '47.45',
   '-0.55',
   '7,502'],
  ['111/01/04',
   '11,582,787',
   '546,668,986',
   '47.50',
   '47.60',
   '47.00',
   '47.30',
   '-0.15',
   '8,038'],
  ['111/01/05',
   '12,283,179',
   '579,369,726',
   '47.10',
   '47.30',
   '47.00',
   '47.15',
   '-0.15',
   '6,147'],
  ['111/01/06',
   '12,468,214',
   '590,435,369',
   '47.30',
   '47.60',
   '47.15',
   '47.60',
   '+0.45',
   '6,576'],
  ['111/01/07',
   '11,036,127',
   '522,523,832',
   '47.60',
   '47.65',
   '47.20',
   '47.45',
   '-0.15',
   '6,945'],
  ['111/01/10',
   '12,136,515',
   '572,075,164',
   '47.45',
   '47.50',
   '47.00',
   '47.30',
   '-0.15',
   '8,558'],
  ['111/01/11',
   '11,147,705',
   '526,597,589',
   '47.10',
   '47.50',
   '47.10'

***
下面列舉了其他crawlers.py中的函數:

* get_stock_id_list( ) *# 取得上市股票名單*

* get_etf_id_list( )   *# 取得etf名單*

* get_otc_id_list( )   *# 取得上櫃股票名單*

* get_0050_list( )     *# 取得0050成份股名單*

* get_latest_trading_days( ) *# 取得目前最新的交易日日期*

* get_stock_month_price(stock_id, year, month) *# 取得股票的某月的股價資料*

***

### crawler_decorator.py

在該檔案中將爬蟲常會用到的功能編寫成裝飾器的形式以方便開發。

以紀錄錯誤訊息為例，在使用爬蟲爬取網站資料的時候常常會遇到連線被拒的情況，這在普通的小實驗中可能沒什麼，但遇到需要大量爬取資料的情況就完全不同了。比如爬取台股資料，中途還好好的，突然在爬取到某隻股票突然報錯，不僅中斷了整套爬取流程，還造成後續收拾的麻煩。

這時候你就可以使用record_error_decorator裝飾器，該裝飾器可以將錯誤訊息保存到指定的檔案中，並回傳一個*run_func_error*類的實例，如果保存錯誤訊息時發生無法寫入檔案的情況，則會回傳一個*record_log_error*類的實例。

下面我們就用一個會隨機拋出錯誤的函數*rand_raise_error*來看看這一切是如何運作的，我們將錯誤訊息保存到rand_func.log檔案中。

In [45]:
from crawler_tool import record_error_decorator
from random import random

# 使用裝飾器將錯誤保存到指定log檔案中
@record_error_decorator(log_path='rand_func.log')
def rand_raise_error():
    n=random()
    if n>0.5:
        # 隨機拋出一個錯誤
        raise ValueError(f"random>0.5")
    return n

然後我們循環呼叫10次，並將*rand_raise_error*的回傳值打印出來，我們會發現當函數沒有報錯時，裝飾器會回傳正確的回傳值，而當報錯時會回傳*run_func_error*。

In [49]:
for i in range(10):
    ret=rand_raise_error()
    print(i, ret)

0 0.09126510786338149
1 <crawler_tool.crawler_decorator.run_func_error object at 0x7f45d6dc9100>
2 0.05924293102474665
3 0.08594711245744047
4 0.40891756028474713
5 0.09208180878285765
6 <crawler_tool.crawler_decorator.run_func_error object at 0x7f45d6b34790>
7 <crawler_tool.crawler_decorator.run_func_error object at 0x7f45d6b34b20>
8 <crawler_tool.crawler_decorator.run_func_error object at 0x7f45d6b341f0>
9 <crawler_tool.crawler_decorator.run_func_error object at 0x7f45d6b34550>


如果要同時辦別回傳的是不是*run_func_error*或是*record_log_error*可以檢查該回傳是不是*decorator_error*的子類。

最後我們將保存到rand_func.log檔案中的錯誤訊息讀取出來。

In [50]:
with open('rand_func.log', 'r') as f:
    for line_str in f.readlines():
        print(line_str)

[20220426 20:18:59] rand_raise_error : random>0.5

[20220426 20:18:59] rand_raise_error : random>0.5

[20220426 20:18:59] rand_raise_error : random>0.5

[20220426 20:18:59] rand_raise_error : random>0.5

[20220426 20:18:59] rand_raise_error : random>0.5



如果我們繼續執行*rand_raise_error*函數，那麼新的錯誤將會被不斷添加到檔案中。

下面是紀錄的格式:

    [紀錄時間] 函數名稱 額外紀錄資訊 : 錯誤訊息
    
你可能會想問**額外紀錄資訊**是什麼？

我們可以通過在函數中加入一個 *_message* 參數來將需要額外紀錄到log檔中的資訊加入到其中。比如我們可以紀錄是跑第幾次發生的錯誤:

In [51]:
for i in range(10):
    ret=rand_raise_error(_message=i)
    print(i, ret)

0 <crawler_tool.crawler_decorator.run_func_error object at 0x7f45d5a64ca0>
1 <crawler_tool.crawler_decorator.run_func_error object at 0x7f45d5a64b20>
2 <crawler_tool.crawler_decorator.run_func_error object at 0x7f45d5a64970>
3 <crawler_tool.crawler_decorator.run_func_error object at 0x7f45d6b348e0>
4 0.32029953539288125
5 0.18570067095780607
6 0.49039423248352043
7 0.24157444876364387
8 <crawler_tool.crawler_decorator.run_func_error object at 0x7f45d6b340d0>
9 0.371367193176363


我們可以發現函數在執行到第0,1,2,3,8次時發生了錯誤，我們可以打開log檔檢查看看是不是如此。

我們可以發現次數的資訊已經被保存到*額外紀錄資訊*的位置上了。

In [52]:
with open('rand_func.log', 'r') as f:
    for line_str in f.readlines():
        print(line_str)

[20220426 20:18:59] rand_raise_error : random>0.5

[20220426 20:18:59] rand_raise_error : random>0.5

[20220426 20:18:59] rand_raise_error : random>0.5

[20220426 20:18:59] rand_raise_error : random>0.5

[20220426 20:18:59] rand_raise_error : random>0.5

[20220426 20:27:05] rand_raise_error 0: random>0.5

[20220426 20:27:05] rand_raise_error 1: random>0.5

[20220426 20:27:05] rand_raise_error 2: random>0.5

[20220426 20:27:05] rand_raise_error 3: random>0.5

[20220426 20:27:05] rand_raise_error 8: random>0.5



***

在crawler_decorator.py還有其他兩種解釋器分別是:

*`try_loop_decorator(times=3, sleep_time=2)`*

    當接受到函數回傳的值為decorator_error的子類時，會暫停sleep_time秒後再重新運行函數，最多times次。如果到最後函數的回傳依然是decorator_error的子類，則直接回傳。

*`backup_decorator(backup_path)`*

    當函數回傳的值為decorator_error的子類時，讀取backup_path路徑中的pickle檔案的內容當作回傳。如果函數回傳的值不是decorator_error的子類時，則將回傳值覆蓋掉backup_path路徑中的pickle檔案的內容。如果backup_path路徑中的pickle檔案不存在則直接建立檔案。
    
恭喜你已經看完全部的內容了，下面是一個爬取0050成分股爬蟲的範例。

***

## 爬取0050成分股爬蟲的範例

In [55]:
from crawler_tool import record_error_decorator
from crawler_tool import backup_decorator
from crawler_tool import get_0050_list

# 成分股名單備份路徑
list_backup_path='backup_0050list.pickle'

# 錯誤訊息保存路徑
error_log_path='0050_list_error.log'

@backup_decorator(backup_path=list_backup_path)
@record_error_decorator(log_path=error_log_path)
def get_stock_list():
    return get_0050_list()